In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [1]:
import torch

device = torch.device('cuda:2')
torch.cuda.set_device(2)
torch.cuda.current_device()

/home/stepan/anaconda3/envs/sol-copilot/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2

In [2]:
torch.cuda.empty_cache()

In [3]:
from datasets import Dataset

ds = Dataset.load_from_disk('/data/solidity-dataset')

In [4]:
ds

Dataset({
    features: ['text'],
    num_rows: 856323
})

In [5]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, TrainingArguments, DataCollatorForLanguageModeling, Trainer
from datasets import Dataset
import evaluate
from itertools import chain
import logging
import numpy as np

logging.getLogger(
    "transformers.tokenization_utils_base").setLevel(logging.ERROR)


def tokenize_function(tokenizer):
    def f(item):
        return tokenizer(item['text'], return_tensors="pt", padding=True)

    return f


BLOCK_SIZE = 512


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {
        k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= BLOCK_SIZE:
        total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE
    # Split by chunks of max_len.
    result = {
        k: [t[i: i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    return result


def prepare_segment(tokenizer: AutoTokenizer, full_ds: Dataset, segment_size: int, segment_i: int):
    segment_ds = Dataset.from_dict(
        full_ds[segment_i * segment_size:(segment_i + 1) * segment_size])

    return segment_ds.map(
        tokenize_function(tokenizer),
        remove_columns='text',
        batched=True,
        batch_size=5,
        num_proc=6,
        cache_file_name=f'/data/solidity-cache/seg_{segment_i}_tokenize_map_cache.cache',
        load_from_cache_file=True,
        desc=f'Segment #{segment_i} | Tokenizing'
    ).map(
        group_texts,
        batched=True,
        batch_size=5,
        num_proc=6,
        drop_last_batch=True,
        cache_file_name=f'/data/solidity-cache/seg_{segment_i}_group_map_cache.cache',
        load_from_cache_file=True,
        desc=f'Segment #{segment_i} | Grouping into batches'
    )


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def train(model, tokenizer, ds: Dataset, segment_size: int, segment_i: int, meta_i: int):
    segment_ds_tokenized = prepare_segment(
        tokenizer, ds, segment_size, segment_i).train_test_split(0.05)

    training_args = TrainingArguments(
        output_dir=f'./results/meta-epoch-{meta_i}/segment-{segment_i}',
        num_train_epochs=1,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        warmup_steps=500,
        save_steps=50000,
        weight_decay=0.01,
        logging_dir=f'./logs/meta-epoch-{meta_i}/segment-{segment_i}',
        logging_steps=10,
        evaluation_strategy='epoch',
        fp16=True,
        learning_rate=1e-5
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=segment_ds_tokenized["train"],
        eval_dataset=segment_ds_tokenized["test"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

In [6]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-large')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-large').to(device)

In [7]:
SEGMENT_SIZE = 10000

In [8]:
train(model, tokenizer, ds, SEGMENT_SIZE, 0, 0)

Epoch,Training Loss,Validation Loss
